# Introduction

In [1]:
import pandas as pd
import numpy as np
import calendar
pd.set_option('display.max_columns', 100)

## Function

### Preprocessing Data

In [2]:
def X_preprocessing(df):
    
    # checking resposne variable
    if 'SalePrice' not in df:
        column_interval_dup = column_interval.copy()
        column_interval_dup.remove('SalePrice')
        
    # drop useless column "id"
    df = df.drop('Id', axis=1)
    
    df_nominal = df[column_nominal].copy()
    df_interval = df[column_interval_dup].copy()

    # preprocessing on interval dataframe
    ## simple null value imputation: interval to 0
    ## concat to the actual table
    df_interval = df_interval.fillna(0)
    df = df.drop(column_interval_dup, axis=1)
    df = pd.concat([df, df_interval], axis=1)

    # binary data convesion
    df['CentralAir'] = df[column_binary]['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)

    # preprocessing on nonimal dataframe
    ## simple null value imputation: categorical to NaN
    df_nominal = df_nominal.fillna('NaN')

    ## convert MSSubClass value to string
    df_nominal['MSSubClass'] = df_nominal['MSSubClass'].astype('string')
    df_nominal['MSSubClass'] = df_nominal['MSSubClass'].apply(lambda x: 'MSSC_'+x)

    ## all set, convert all the categorical variable to dummies variables
    df_nominal = pd.get_dummies(df_nominal)

    ## concat to the actual table
    df = df.drop(column_nominal, axis=1)
    df = pd.concat([df, df_nominal], axis=1)

    # preprocessing on time dataframe
    ## time approach-renovation: comparing to the year built
    ## time approach-soldyear: comparing to the year built
    df['YearRemodAdd'] = df[column_time]['YearRemodAdd'] - df[column_time]['YearBuilt'] 
    df['YrSold'] = df[column_time]['YrSold'] - df[column_time]['YearBuilt']

    ## time approach-built: comparing to the oldest listing on the market
    df_yrbuilt_min = df[column_time]['YearBuilt'].min()
    df['YearBuilt'] = df[column_time]['YearBuilt'].apply(lambda x: x - df_yrbuilt_min)

    ## time apprach-soldmonth: to dummies variable
    df['MoSold'] = df['MoSold'].apply(lambda x: 'sold_'+calendar.month_name[x])
    df_dum_months = pd.get_dummies(df['MoSold'])
    df = df.drop(['MoSold'], axis=1)

    df = pd.concat([df,df_dum_months], axis=1)

    if 'SalePrice' in df:
    # final to split between dependent variable to resposne
        X = df.drop('SalePrice', axis=1)
        y = df['SalePrice']

    else:
        print('This is test dataset return only X...')
        X = df.copy()
        y = None

    return X, y

In [3]:
def z_score_scaling(X_df, mean, stdev):
    '''
    This function converts the normal data after its pass preprocessing to z-score scalling
    input: x df, mean, and stdev
    output: x df (scalled)
    '''
    
    return 

## Import data(s)

In [4]:
train_df = pd.read_csv('data/train.csv')
df_cols_process = pd.read_csv('data/column_types_and_process_imputation.csv')

## Data Exploration

-> Numerical - Categorical (two general groups) <- 

- Useless (unique identifier usually useless)
- Nominal (as named or categorical)
- Binary (either 1/0)
- Ordinal (Ordinal)
- Count (Integer number starting from 0 exclude negative number)
- Time (consiting seasonal and date)
- Interval (Examples include percentages, temperatures, and income.)

In [5]:
df_cols_process['data types'] = df_cols_process['data types'].apply(lambda x: x.strip())

In [6]:
df_cols_process['data types'].unique()

array(['useless', 'nominal', 'interval', 'ordinal', 'time', 'binary'],
      dtype=object)

In [7]:
column_interval = list(df_cols_process[df_cols_process['data types'] == 'interval']['column']) # no need processing
column_nominal = list(df_cols_process[df_cols_process['data types'] == 'nominal']['column']) 
column_ordinal = list(df_cols_process[df_cols_process['data types'] == 'ordinal']['column']) # no need processing
column_binary = list(df_cols_process[df_cols_process['data types'] == 'binary']['column'])
column_time = list(df_cols_process[df_cols_process['data types'] == 'time']['column'])

## Data Pre-processing

### Drop useless data types

In [8]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# from sklearn.preprocessing import OrdinalEncoder

In [9]:
train_df_temp = train_df.drop('Id',axis = 1).copy()

In [10]:
train_df_nominal = train_df_temp[column_nominal].copy()
train_df_interval = train_df_temp[column_interval].copy()

### Data Imputation

#### Nominal

In [43]:
# build an array for removing nominal columns that have missinv value more than 30%.
col_major_missing = np.array(train_df_nominal.columns[train_df_nominal.isnull().sum()/train_df_nominal.shape[0] > 0.30])

# drop it
train_df_nominal = train_df_nominal.drop(col_major_missing, axis=1)

# assign mode to a variable "nominal mode".
nominal_mode = train_df_nominal.mode().iloc[0]

# simple null value imputation: Mode
train_df_nominal = train_df_nominal.fillna(nominal_mode)

#### Interval 

In [44]:
# simple null value imputation: interval to 0
train_df_interval = train_df_interval.fillna(0)

In [45]:
train_df_temp = train_df_temp.drop(column_interval, axis=1)
train_df_temp = pd.concat([train_df_temp, train_df_interval], axis=1)

### Nominal data to Dummies

In [46]:
train_df_nominal['MSSubClass'] = train_df_nominal['MSSubClass'].astype('string')
train_df_nominal['MSSubClass'] = train_df_nominal['MSSubClass'].apply(lambda x: 'MSSC_'+x)

In [47]:
# from 43 to 281 columns
train_df_nominal = pd.get_dummies(train_df_nominal)

In [48]:
train_df_temp = train_df_temp.drop(column_nominal, axis=1)
train_df_temp = pd.concat([train_df_temp, train_df_nominal], axis=1)

### Change the value of binary data

In [49]:
train_df_temp['CentralAir'] = train_df_temp[column_binary]['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)

### Change the value of time data

In [50]:
# unique approach on how to process the time data
train_df_temp['YearRemodAdd'] = train_df_temp[column_time]['YearRemodAdd'] - train_df_temp[column_time]['YearBuilt']
train_df_temp['YrSold'] = train_df_temp[column_time]['YrSold'] - train_df_temp[column_time]['YearBuilt']

train_df_temp_yrbuilt_min = train_df_temp[column_time]['YearBuilt'].min()
train_df_temp['YearBuilt'] = train_df_temp[column_time]['YearBuilt'].apply(lambda x: x - train_df_temp_yrbuilt_min)

In [51]:
# turn YrSold compare to Yr Built.
train_df_temp['MoSold'] = train_df_temp['MoSold'].apply(lambda x: 'sold_'+calendar.month_name[x])
train_df_temp_dum_months = pd.get_dummies(train_df_temp['MoSold'])
train_df_temp = train_df_temp.drop(['MoSold'], axis=1)

train_df_temp = pd.concat([train_df_temp,train_df_temp_dum_months], axis=1)

## Define Dependent and Response Data of Training

In [52]:
X_train = train_df_temp.drop('SalePrice', axis=1)
Y_train = train_df_temp['SalePrice']

---

## Feature Scalling

### Z-score

In [53]:
# Zscore using SKLearn

from sklearn.preprocessing import scale
# scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)

def zscore_normalize_features(X, mu = False, sigma = False):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    
    if mu is False and sigma is False:
        # find the mean of each column/feature
        mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
        # find the standard deviation of each column/feature
        sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
        # element-wise, subtract mu for that column from each example, divide by std for that column
        
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [54]:
# using SKLearn
# X_norm = scale(X_train, with_mean = True, with_std = True, copy = True)
X_norm, mu, sigma = zscore_normalize_features(X_train)

## Fitting Linear Regression

In [55]:
from sklearn.linear_model import LinearRegression

In [56]:
linear_model = LinearRegression()

In [57]:
X_norm_arr = np.array(X_norm)

In [59]:
linear_model.fit(X_norm_arr, Y_train)

LinearRegression()

In [60]:
b = linear_model.intercept_
w = linear_model.coef_

In [61]:
Y_train_pred = linear_model.predict(X_norm)

## Root Mean Squared Error

In [62]:
from sklearn.metrics import mean_squared_error
import math

In [63]:
train_eval_metrics = math.sqrt(mean_squared_error(Y_train_pred, Y_train))

In [64]:
train_eval_metrics

20949.493465736316

## Test Dataset

In [65]:
test_df = pd.read_csv('data/test.csv')

In [74]:
X_test, y = X_preprocessing(test_df)

This is test dataset return only X...


In [107]:
X_test[X_test['OverallQual'].isna()]

,OverallQual,OverallCond,YearBuilt,YearRemodAdd,CentralAir,YrSold,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_MSSC_120,MSSubClass_MSSC_150,MSSubClass_MSSC_160,MSSubClass_MSSC_180,MSSubClass_MSSC_190,MSSubClass_MSSC_20,MSSubClass_MSSC_30,MSSubClass_MSSC_40,MSSubClass_MSSC_45,MSSubClass_MSSC_50,MSSubClass_MSSC_60,MSSubClass_MSSC_70,MSSubClass_MSSC_75,MSSubClass_MSSC_80,MSSubClass_MSSC_85,...,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NaN,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Gd,PoolQC_NaN,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NaN,MiscFeature_Gar2,MiscFeature_NaN,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_NaN,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,sold_April,sold_August,sold_December,sold_February,sold_January,sold_July,sold_June,sold_March,sold_May,sold_November,sold_October,sold_September


In [112]:
X_test

,OverallQual,OverallCond,YearBuilt,YearRemodAdd,CentralAir,YrSold,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_MSSC_120,MSSubClass_MSSC_150,MSSubClass_MSSC_160,MSSubClass_MSSC_180,MSSubClass_MSSC_190,MSSubClass_MSSC_20,MSSubClass_MSSC_30,MSSubClass_MSSC_40,MSSubClass_MSSC_45,MSSubClass_MSSC_50,MSSubClass_MSSC_60,MSSubClass_MSSC_70,MSSubClass_MSSC_75,MSSubClass_MSSC_80,MSSubClass_MSSC_85,...,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NaN,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Gd,PoolQC_NaN,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NaN,MiscFeature_Gar2,MiscFeature_NaN,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_NaN,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,sold_April,sold_August,sold_December,sold_February,sold_January,sold_July,sold_June,sold_March,sold_May,sold_November,sold_October,sold_September
0,5,6,82,0,1,49,80.0,11622,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,6,6,79,0,1,52,81.0,14267,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
2,5,5,118,1,1,13,74.0,13830,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,6,6,119,0,1,12,78.0,9978,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,8,5,113,0,1,18,43.0,5005,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,4,7,91,0,1,36,21.0,1936,0.0,0.0,0.0,546.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,5,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1455,4,5,91,0,1,36,21.0,1894,0.0,252.0,0.0,294.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,6,0,1970.0,1.0,286.0,0,24,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1456,5,7,81,36,1,46,160.0,20000,0.0,1224.0,0.0,0.0,1224.0,1224,0,0,1224,1.0,0.0,1,0,4,1,7,1,1960.0,2.0,576.0,474,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1457,5,5,113,0,1,14,62.0,10441,0.0,337.0,0.0,575.0,912.0,970,0,0,970,0.0,1.0,1,0,3,1,6,0,0.0,0.0,0.0,80,32,0,0,0,0,700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0

In [113]:
X_train

,OverallQual,OverallCond,YearBuilt,YearRemodAdd,CentralAir,YrSold,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_MSSC_120,MSSubClass_MSSC_160,MSSubClass_MSSC_180,MSSubClass_MSSC_190,MSSubClass_MSSC_20,MSSubClass_MSSC_30,MSSubClass_MSSC_40,MSSubClass_MSSC_45,MSSubClass_MSSC_50,MSSubClass_MSSC_60,MSSubClass_MSSC_70,MSSubClass_MSSC_75,MSSubClass_MSSC_80,MSSubClass_MSSC_85,MSSubClass_MSSC_90,...,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NaN,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_NaN,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NaN,MiscFeature_Gar2,MiscFeature_NaN,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,sold_April,sold_August,sold_December,sold_February,sold_January,sold_July,sold_June,sold_March,sold_May,sold_November,sold_October,sold_September
0,7,5,131,0,1,5,65.0,8450,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,6,8,104,0,1,31,80.0,9600,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,7,5,129,1,1,7,68.0,11250,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,7,5,43,55,1,91,60.0,9550,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,8,5,128,0,1,8,84.0,14260,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,5,127,1,1,8,62.0,7917,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1456,6,6,106,10,1,32,85.0,13175,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1457,7,9,69,65,1,69,66.0,9042,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1458,5,6,78,46,1,60,68.0,9717,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [115]:
X_test_scale, mu_test, sigma_test = zscore_normalize_features(X_test, mu, sigma)

In [126]:
X_test_scale.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley_Grvl,Alley_NaN,Alley_Pave,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NaN,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NaN,BsmtExposure_No,BsmtFinSF1,BsmtFinSF2,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NaN,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NaN,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFullBath,BsmtHalfBath,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NaN,BsmtQual_TA,BsmtUnfSF,CentralAir,Condition1_Artery,Condition1_Feedr,Condition1_Norm,...,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_NaN,SaleType_New,SaleType_Oth,SaleType_WD,ScreenPorch,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NaN,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,sold_April,sold_August,sold_December,sold_February,sold_January,sold_July,sold_June,sold_March,sold_May,sold_November,sold_October,sold_September
0,-0.689929,-0.795163,-0.116339,-0.188311,0.257821,-0.169981,-1.062465,0.443533,-0.147287,-0.192177,-0.1742,-0.291025,-0.178331,-0.215859,-0.16125,-0.037037,0.337126,-0.422338,-0.317893,-0.291025,-0.163471,0.729386,0.053428,0.604293,-0.421212,-0.335864,-0.633365,-0.231065,-0.16125,3.158709,-0.646124,-0.114827,-0.152071,-0.098397,5.544288,-0.163471,-0.195977,-2.481303,-0.819964,-0.241061,-0.300609,-0.156721,-0.856719,-0.16125,1.117862,-0.672923,0.263813,-0.184376,4.126099,-2.50998,...,NaN,-0.087129,-0.058621,-0.064238,-0.094785,0.528753,-0.087129,-0.493570,-0.069409,-0.037037,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,-0.1742,-0.052414,-0.037037,-0.078757,-0.058621,-0.058621,NaN,-0.301962,-0.045376,0.390293,1.882709,-0.064238,0.064238,-0.934130,-0.400017,0.02618,NaN,NaN,0.365179,-0.571923,-0.555742,0.411777,-0.326954,-0.301962,-0.205214,-0.192177,-0.203395,-0.436881,2.184205,-0.279797,-0.403014,-0.239176,-0.254786,-0.21236
1,0.430511,-0.795163,-0.116339,-0.188311,0.257821,-0.169981,0.163779,0.443533,-0.147287,-0.192177,-0.1742,-0.291025,-0.178331,-0.215859,-0.16125,-0.037037,0.337126,-0.422338,-0.317893,-0.291025,-0.163471,0.729386,1.051363,-0.288653,2.374103,-0.335864,-0.633365,-0.231065,-0.16125,-0.316585,-0.646124,-0.114827,-0.152071,-0.098397,-0.180366,-0.163471,-0.195977,0.403014,-0.819964,-0.241061,-0.300609,-0.156721,-0.856719,-0.16125,1.117862,-0.365032,0.263813,-0.184376,-0.242360,0.39841,...,NaN,-0.087129,-0.058621,-0.064238,-0.094785,-1.891244,-0.087129,2.026054,-0.069409,-0.037037,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,-0.1742,-0.052414,-0.037037,-0.078757,-0.058621,-0.058621,NaN,-0.301962,-0.045376,0.390293,-0.270208,-0.064238,0.064238,-0.318683,0.619239,0.02618,NaN,NaN,2.384400,-0.671285,-0.555742,0.510984,-0.326954,-0.301962,-0.205214,-0.192177,-0.203395,-0.436881,2.184205,-0.279797,-0.403014,-0.239176,-0.254786,-0.21236
2,-0.607125,0.811239,-0.116339,-0.188311,0.257821,-0.169981,0.163779,0.443533,-0.147287,-0.192177,-0.1742,-0.291025,-0.178331,-0.215859,-0.16125,-0.037037,0.337126,-0.422338,-0.317893,-0.291025,-0.163471,0.729386,0.761852,-0.288653,-0.421212,-0.335864,1.578868,-0.231065,-0.16125,-0.316585,-0.646124,-0.114827,-0.152071,-0.098397,-0.180366,-0.163471,-0.195977,0.403014,-0.819964,-0.241061,-0.300609,-0.156721,1.167244,-0.16125,-0.894565,-0.974021,0.263813,-0.184376,-0.242360,0.39841,...,NaN,-0.087129,-0.058621,-0.064238,-0.094785,0.528753,-0.087129,-0.493570,-0.069409,-0.037037,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,-0.1742,-0.052414,-0

In [131]:
X_test_scale[['Condition2_RRAe']]

,Condition2_RRAe
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1454,NaN
1455,NaN
1456,NaN
1457,NaN


In [133]:
X_train['Utilities_NaN']

KeyError: 'Utilities_NaN'

In [135]:
X_train['Exterior2nd']

KeyError: 'Exterior2nd'

In [130]:
X_test_scale.columns[X_test_scale.isnull().sum() != 0]

Index(['Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'Electrical_Mix', 'Electrical_NaN', 'Exterior1st_ImStucc',
       'Exterior1st_NaN', 'Exterior1st_Stone', 'Exterior2nd_NaN',
       'Exterior2nd_Other', 'Functional_NaN', 'GarageQual_Ex', 'Heating_Floor',
       'Heating_OthW', 'HouseStyle_2.5Fin', 'KitchenQual_NaN',
       'MSSubClass_MSSC_150', 'MSZoning_NaN', 'MiscFeature_TenC', 'PoolQC_Fa',
       'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal',
       'RoofMatl_Roll', 'SaleType_NaN', 'Utilities_NaN', 'Utilities_NoSeWa'],
      dtype='object')

# Resources

https://medium.com/analytics-vidhya/ways-to-handle-categorical-column-missing-data-its-implementations-15dc4a56893#:~:text=Step%201%3A%20Find%20which%20category,and%20keep%20newly%20imputed%20columns.&text=Advantage%3A%20Simple%20and%20easy%20to%20implement%20for%20categorical%20variables%2Fcolumns.

